# Cross-Team Semantic Table Example

This example demonstrates how different teams can define their own semantic tables and combine them to create cross-functional metrics.

In [1]:
from boring_semantic_layer.semantic_api import to_semantic_table
import ibis
import pandas as pd

## Setup: Create Mock Data

Let's create sample data for marketing's Users table and support's SupportCases table.

In [2]:
# Marketing team's user data
users_data = pd.DataFrame({
    'customer_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'segment': ['Enterprise', 'SMB', 'Enterprise', 'SMB', 'Enterprise', 
                'SMB', 'Enterprise', 'SMB', 'Enterprise', 'SMB'],
    'signup_date': pd.to_datetime(['2023-01-15', '2023-02-20', '2023-01-10', 
                                   '2023-03-05', '2023-02-15', '2023-03-20',
                                   '2023-01-25', '2023-02-28', '2023-03-10',
                                   '2023-03-15']),
    'monthly_spend': [5000, 500, 8000, 300, 6000, 400, 7000, 600, 9000, 350]
})

# Support team's case data
support_cases_data = pd.DataFrame({
    'case_id': range(1, 26),
    'customer_id': [1, 1, 1, 2, 3, 3, 3, 3, 4, 5, 5, 6, 7, 7, 8, 8, 8, 9, 9, 10,
                    1, 3, 5, 7, 9],
    'created_date': pd.to_datetime(['2023-02-01', '2023-02-15', '2023-03-01',
                                    '2023-03-10', '2023-02-05', '2023-02-20',
                                    '2023-03-05', '2023-03-15', '2023-03-20',
                                    '2023-03-01', '2023-03-10', '2023-03-25',
                                    '2023-02-10', '2023-03-05', '2023-03-15',
                                    '2023-03-18', '2023-03-22', '2023-03-12',
                                    '2023-03-20', '2023-03-25', '2023-03-28',
                                    '2023-03-30', '2023-04-01', '2023-04-02',
                                    '2023-04-03']),
    'priority': ['High', 'Medium', 'Low', 'High', 'High', 'High', 'Medium', 'Low',
                 'Medium', 'High', 'Medium', 'Low', 'High', 'Medium', 'High',
                 'Medium', 'Low', 'High', 'Medium', 'Low', 'High', 'High',
                 'Medium', 'Low', 'High']
})

# Create Ibis tables
users_table = ibis.memtable(users_data, name='users')
support_cases_table = ibis.memtable(support_cases_data, name='support_cases')

## Step 1: Marketing Team Defines Users Semantic Table

In [3]:
# Marketing team defines their semantic table
marketing_st = (
    to_semantic_table(users_table, name="users")
    .with_dimensions(
        customer_id=lambda t: t.customer_id,
        segment=lambda t: t.segment,
        signup_date=lambda t: t.signup_date
    )
    .with_measures(
        user_count=lambda t: t.customer_id.count(),
        total_revenue=lambda t: t.monthly_spend.sum(),
        avg_revenue_per_user=lambda t: t.monthly_spend.mean()
    )
)

# Marketing team can query their metrics
marketing_metrics = (
    marketing_st
    .group_by("segment")
    .aggregate(
        user_count=lambda t: t.user_count,
        total_revenue=lambda t: t.total_revenue,
        avg_revenue_per_user=lambda t: t.avg_revenue_per_user
    )
)
print("Marketing Metrics by Segment:")
print(marketing_metrics.execute())

Marketing Metrics by Segment:
      segment  user_count  total_revenue  avg_revenue_per_user
0         SMB           5           2150                 430.0
1  Enterprise           5          35000                7000.0


## Step 2: Support Team Defines SupportCases Semantic Table

In [4]:
# Support team defines their semantic table
support_st = (
    to_semantic_table(support_cases_table, name="support_cases")
    .with_dimensions(
        case_id=lambda t: t.case_id,
        customer_id=lambda t: t.customer_id,
        created_date=lambda t: t.created_date,
        priority=lambda t: t.priority
    )
    .with_measures(
        case_count=lambda t: t.case_id.count(),
        high_priority_cases=lambda t: (t.priority == 'High').sum()
    )
)

# Support team can query their metrics
support_metrics = (
    support_st
    .group_by("priority")
    .aggregate(case_count=lambda t: t.case_count)
)
print("\nSupport Metrics by Priority:")
print(support_metrics.execute())


Support Metrics by Priority:
  priority  case_count
0      Low           6
1   Medium           8
2     High          11


## Step 3: Join Tables and Create Cross-Team Semantic Table

Now we combine both semantic tables and create a new semantic table with cross-team metrics.

In [5]:
# Join the semantic tables
joined_st = marketing_st.join(support_st, on=lambda u, s: u.customer_id == s.customer_id)

# Create a new semantic table with cross-team metrics
# Note: We need to explicitly define which dimensions we want to include
cross_team_st = (
    to_semantic_table(joined_st, name="cross_team_metrics")
    .with_dimensions(
        # Include key dimensions from both tables
        users__segment=lambda t: t.segment,
        users__customer_id=lambda t: t.customer_id,
        support_cases__priority=lambda t: t.priority,
        support_cases__created_date=lambda t: t.created_date
    )
    .with_measures(
        # Include original measures from both tables
        users__user_count=lambda t: t.users__user_count,
        users__total_revenue=lambda t: t.users__total_revenue,
        support_cases__case_count=lambda t: t.support_cases__case_count,
        support_cases__high_priority_cases=lambda t: t.support_cases__high_priority_cases,
        
        # New cross-team measures that combine data from both teams
        cases_per_user=lambda t: t.support_cases__case_count / t.users__user_count,
        support_cost_ratio=lambda t: t.support_cases__case_count / t.users__total_revenue * 1000,  # cases per $1000 revenue
        high_priority_rate=lambda t: t.support_cases__high_priority_cases / t.support_cases__case_count,
        avg_case_value=lambda t: t.users__total_revenue / t.support_cases__case_count,  # revenue per case
    )
)

print("Cross-team semantic table created with new measures!")

Cross-team semantic table created with new measures!


In [6]:
cross_team_st.dimensions,
cross_team_st.measures

{'cross_team_metrics__users__user_count': Measure(expr=<function <lambda> at 0x10bfe1da0>, description=None),
 'cross_team_metrics__users__total_revenue': Measure(expr=<function <lambda> at 0x10bfe1d00>, description=None),
 'cross_team_metrics__support_cases__case_count': Measure(expr=<function <lambda> at 0x10bfe1bc0>, description=None),
 'cross_team_metrics__support_cases__high_priority_cases': Measure(expr=<function <lambda> at 0x10bfe19e0>, description=None),
 'cross_team_metrics__cases_per_user': Measure(expr=<function <lambda> at 0x10bfe0b80>, description=None),
 'cross_team_metrics__support_cost_ratio': Measure(expr=<function <lambda> at 0x10bfe1e40>, description=None),
 'cross_team_metrics__high_priority_rate': Measure(expr=<function <lambda> at 0x10bfe1ee0>, description=None),
 'cross_team_metrics__avg_case_value': Measure(expr=<function <lambda> at 0x10bfe1f80>, description=None)}

## Step 4: Query Cross-Team Metrics

The user or LLM can now query these combined metrics without worrying about the underlying data sources.

In [11]:
# Query 1: Cases per user by customer segment using the new measure
segment_support_metrics = (
    cross_team_st
    .group_by("segment")
    .aggregate(
        user_count=lambda t: t.users__user_count,
        case_count=lambda t: t.support_cases__case_count,
        cases_per_user=lambda t: t.cases_per_user  # Using the new predefined measure
    )
)

print("Support Cases per Customer Segment:")
print(segment_support_metrics.execute())

Support Cases per Customer Segment:


AttributeError: 'Table' object has no attribute 'users__user_count'

In [12]:
# Query 2: Support efficiency metrics using the new cross-team measures
efficiency_metrics = (
    cross_team_st
    .group_by("users__segment")
    .aggregate(
        total_revenue=lambda t: t.users__total_revenue,
        case_count=lambda t: t.support_cases__case_count,
        support_cost_ratio=lambda t: t.support_cost_ratio,  # Using new measure
        high_priority_rate=lambda t: t.high_priority_rate,  # Using new measure
        avg_case_value=lambda t: t.avg_case_value  # Using new measure
    )
)

print("\nSupport Efficiency by Customer Segment:")
print(efficiency_metrics.execute())


Support Efficiency by Customer Segment:


AttributeError: 'Table' object has no attribute 'users__total_revenue'

In [14]:
# Query 3: Priority analysis with new measures
priority_metrics = (
    cross_team_st
    .group_by("support_cases__priority")
    .aggregate(
        case_count=lambda t: t.support_cases__case_count,
        avg_case_value=lambda t: t.avg_case_value,
        total_revenue=lambda t: t.users__total_revenue
    )
)

print("\nCase Value by Priority:")
print(priority_metrics.execute())


Case Value by Priority:


AttributeError: 'Table' object has no attribute 'support_cases__case_count'